# Visualizing Nonbinary Trees

This example lets you visualize the hierarchy of nonbinary trees.

***Put example image here?***

In [244]:
def n_by_1_empty_list(n):
    """
    Create an n-by-1 empty list, e.g. n=3 produces [[], [], []]

    :param n: The number of empty (sub-)lists to include in the parent list
    """
    n_by_1_list = []
    for i in range(n):
        # n_by_1_list.extend([[]])
        n_by_1_list += [[]]

    return n_by_1_list

def r_by_c_empty_list(r, c):
    matrix = []
    for row_index in range(r):
        one_row = n_by_1_empty_list(c)
        # matrix.extend([one_row])
        matrix += [one_row]
        
    return matrix

def r_by_c_empty_string_list(r, c):
    matrix = []
    for row_index in range(r):
        one_row = [""] * c
        matrix += [one_row]
        
    return matrix

def interleave(a, b):
    len_b = len(b)
    interleaved = []
    for row_index, row in enumerate(a):
        interleaved += [row]
        # If b hasn't run out of rows, insert corresponding row of b
        if row_index < len_b:
            interleaved += [b[row_index]]
    return interleaved

In [245]:
def concat_grids_horizontally(grid1:list[list[str]], grid2:list[list[str]]) -> list[list[str]]:
    """Concatenate two nested lists horizontally, for example
    inputs [['a'],['b'],['c']] and [['d'], ['e'], ['f']] 
    produce [['a', 'd'], ['b', 'e'], ['c', 'f']]

    :returns: The confined grid, a two-deep nested list of strings
    :param grid1: The first grid, a two-deep nested list of strings
    :param grid2: The second grid, a two-deep nested list of strings
    """
    if grid1 == [[]]:
        combined = grid2
    elif grid2 == [[]]:
        combined = grid1
    else:
        combined = []
        for row_counter in range(len(grid1)):
            combined += [grid1[row_counter] + grid2[row_counter]]
    return combined

class NonBinTree:
    """
    Nonbinary tree class
    Note that this class is not designed to sort nodes as they are added to the tree;
    the assumption is that they should be ordered in the order added
    Adapted from https://stackoverflow.com/questions/60579330/non-binary-tree-data-structure-in-python#60579464
    """

    def __init__(self, val:str):
        """Create a NonBinTree instance"""
        self.val = val
        self.nodes = []

    def add_node(self, val:str):
        """Add a node to the tree and return the new node"""
        self.nodes.append(NonBinTree(val))
        return self.nodes[-1]

    def __repr__(self) -> str:
        """Print out the tree as a nested list"""
        return f"NonBinTree({self.val}): {self.nodes}"

    def get_ncols(self) -> int:
        """Get the number of columns in the tree"""
        self.ncols = 0
        if len(self.nodes) > 0:
            # If there are nodes under this one, call get_ncols on them recursively
            for node in self.nodes:
                self.ncols += node.get_ncols()
        else:
            # If there are no nodes under this one, add 1 for this node
            self.ncols += 1
        return self.ncols

    def get_max_depth(self) -> int:
        """Get the maximum depth of the tree"""
        max_depth = 0
        if len(self.nodes) > 0:
            for node in self.nodes:
                this_depth = node.get_max_depth()
                max_depth = max(this_depth + 1, max_depth)
        else:
            max_depth = max(1, max_depth)
        self.max_depth = max_depth
        return self.max_depth

    def get_grid(self) -> list[list[str]]:
        """
        Get a two-dimensional grid where
        each row is a level in the fragment hierarchy, and
        the columns serve to arrange the fragments horizontally
        """
        # Call methods to calculate self.ncols and self.max_depth
        self.get_ncols()
        self.get_max_depth()

        # Create top row: Node value, then the rest of columns are blank (empty strings)
        self.grid = [[self.val] + [""] * (self.ncols - 1)]

        n_nodes = len(self.nodes)

        if n_nodes > 0:
            nodes_grid = [[]]

            # Iterate through the chile nodes
            for node_counter, node in enumerate(self.nodes):
                # Recursively call this function to get the grid for children
                node_grid = node.get_grid()

                # Add spacer rows if needed
                node_grid_rows = len(node_grid)
                rows_padding = self.max_depth - node_grid_rows - 1
                for padding in range(rows_padding):
                    node_grid += [[""] * len(node_grid[0])]

                nodes_grid = concat_grids_horizontally(nodes_grid, node_grid)

            self.grid += nodes_grid

        return self.grid

    def get_tree(self) -> str:
        """
        Get a visual hierarchy tree
        """
        # Call method to create the grid, a list of list of str
        self.get_grid()

        # print(f"self.grid: {len(self.grid)} x {len(self.grid[0])}")

        markers = r_by_c_empty_string_list(len(self.grid) - 1, len(self.grid[0]))
        # print(f"{markers=}")
        # print(f"markers: {len(markers)} x {len(markers[0])}")
        
        # Iterate from rows to from #2 (index = 1) to last, 
        #   because there is nothing above the top row
        for row_index, row in enumerate(self.grid):
            if row_index > 0:
                # print(f"{row_index}: {row}. {len(row)=}")
                for col_index, col in enumerate(row):
                    # print(f"{row_index=} {col_index=}: {col}, {len(col)=}")
                    if len(col) > 0:
                        markers[row_index - 1][col_index] = "|"
                    # print(f"{markers=}")
        return interleave(self.grid, markers)


In [246]:
a = [["hi", "there"], ["what", "time"]]
b = [["howdy", "back"], ["is", "it"]]
interleave(a, b)

[['hi', 'there'], ['howdy', 'back'], ['what', 'time'], ['is', 'it']]

In [247]:
quadrilateral = NonBinTree("quadrilateral")
trapezoid = quadrilateral.add_node("trapezoid")
isosceles_trapezoid = trapezoid.add_node("isosceles trapezoid")
parallelogram = quadrilateral.add_node("parallelogram")
rhombus = parallelogram.add_node("rhombus")
rhombus.add_node("square")
rectangle = parallelogram.add_node("rectangle")
rectangle.add_node("square")
kite = quadrilateral.add_node("kite")

In [248]:
quadrilateral.get_grid()

[['quadrilateral', '', '', ''],
 ['trapezoid', 'parallelogram', '', 'kite'],
 ['isosceles trapezoid', 'rhombus', 'rectangle', ''],
 ['', 'square', 'square', '']]

```
'quadrilateral       ,  ''            , ''         , ''
'trapezoid'          , 'parallelogram', ''         , 'kite'
'isosceles trapezoid', 'rhombus'      , 'rectangle', ''
''                   , 'square'       , 'square'   , ''
```

In [249]:
quadrilateral.get_tree()

[['quadrilateral', '', '', ''],
 ['|', '|', '', '|'],
 ['trapezoid', 'parallelogram', '', 'kite'],
 ['|', '|', '|', ''],
 ['isosceles trapezoid', 'rhombus', 'rectangle', ''],
 ['', '|', '|', ''],
 ['', 'square', 'square', '']]

```
 ['quadrilateral'      , ''             , ''         , ''    ],
 ['|'                  , '|'            , ''         , '|'   ],
 ['trapezoid'          , 'parallelogram', ''         , 'kite'],
 ['|'                  , '|'            , '|'        , ''    ],
 ['isosceles trapezoid', 'rhombus'      , 'rectangle', ''    ],
 [''                   , '|'            , '|'        , ''    ],
 [''                   , 'square'       , 'square'   , ''    ]
```